In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

os.chdir('/Users/mac/Desktop/regression')

2023-04-23 09:29:14.804906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_comment_opinion = pd.read_csv('df_all_predict_0422.csv', encoding='ISO-8859-1')

In [4]:
# drop rows with stock = "A"
df_comment_opinion = df_comment_opinion[df_comment_opinion['stock'] != 'A']

# modify the 'label' column using loc indexer
df_comment_opinion.loc[df_comment_opinion['label'] == 2, 'label'] = -1
# Filter out the rows with category=1
df_comment = df_comment_opinion[df_comment_opinion['category']== '1']
df_opinion = df_comment_opinion[df_comment_opinion['category']== '2']

In [5]:
df_comment['label'] = df_comment['label'].astype(int)
df_opinion['label'] = df_opinion['label'].astype(int)

In [6]:
def calculate_weighted_sentiment_index(df, penalty_alpha=0.5):
    weighted_sentiment_indices = (
        df.groupby(["stock", "Date"])["label"]
        .agg(["sum", "count"])
        .reset_index()
    )
    weighted_sentiment_indices["penalty"] = penalty_alpha / weighted_sentiment_indices["count"]
    weighted_sentiment_indices["weighted_sentiment_index"] = (
        weighted_sentiment_indices["sum"] / weighted_sentiment_indices["count"]
    ) * (1 - weighted_sentiment_indices["penalty"])

    return weighted_sentiment_indices[["stock", "Date", "weighted_sentiment_index", "count"]]
weighted_comment_indices = calculate_weighted_sentiment_index(df_comment)
print(weighted_comment_indices)

     stock        Date  weighted_sentiment_index  count
0     AAPL  2022/01/01                  0.656250      4
1     AAPL  2022/01/02                  0.750000      2
2     AAPL  2022/01/04                  1.222222      6
3     AAPL  2022/01/05                  0.530612      7
4     AAPL  2022/01/06                  0.750000      2
...    ...         ...                       ...    ...
2757    ZM  2022/11/12                  1.000000      1
2758    ZM  2022/11/23                  1.000000      1
2759    ZM  2022/12/28                  0.500000      1
2760    ZM  2023/01/18                  0.500000      1
2761    ZM  2023/02/17                  1.000000      1

[2762 rows x 4 columns]


In [7]:
weighted_opinion_indices = calculate_weighted_sentiment_index(df_opinion)
print(weighted_opinion_indices)

     stock        Date  weighted_sentiment_index  count
0     AAPL  2022/01/01                  0.000000      1
1     AAPL  2022/01/02                  0.000000      1
2     AAPL  2022/01/03                  0.524691      9
3     AAPL  2022/01/04                  0.360000      5
4     AAPL  2022/01/05                  0.375000      2
...    ...         ...                       ...    ...
1766    ZM  2022/11/22                  0.750000      2
1767    ZM  2022/11/28                  0.500000      1
1768    ZM  2022/11/29                  1.500000      2
1769    ZM  2022/12/19                  0.000000      1
1770    ZM  2023/02/10                  0.500000      1

[1771 rows x 4 columns]


In [8]:
def reformat_sentiment_indices(weighted_sentiment_indices):
    sentiment_time_series = weighted_sentiment_indices.pivot(
        index="Date", columns="stock", values="weighted_sentiment_index"
    )
    count_time_series = weighted_sentiment_indices.pivot(
        index="Date", columns="stock", values="count"
    )
    count_time_series = count_time_series.add_suffix("_count")
    sentiment_time_series = pd.concat([sentiment_time_series, count_time_series], axis=1)
    
    # Reset the index to move the "date" index into a regular column
    sentiment_time_series = sentiment_time_series.reset_index()
    
    return sentiment_time_series
comment_time_series = reformat_sentiment_indices(weighted_comment_indices)
comment_time_series

stock        Date      AAPL  AMC     AMD      AMZN  AVGO  BABA    BB  \
0      2021/12/31       NaN  NaN     NaN       NaN   NaN   NaN   NaN   
1      2022/01/01  0.656250  NaN  0.4375  1.364198   NaN   1.0   NaN   
2      2022/01/02  0.750000  NaN     NaN       NaN   NaN   1.0   NaN   
3      2022/01/03       NaN  NaN     NaN       NaN   NaN   NaN   NaN   
4      2022/01/04  1.222222  NaN  0.0000  0.500000   NaN   0.5  0.75   
..            ...       ...  ...     ...       ...   ...   ...   ...   
415    2023/02/20       NaN  NaN  1.1250       NaN   NaN   NaN   NaN   
416    2023/02/21       NaN  NaN     NaN       NaN   NaN   NaN   NaN   
417    2023/02/22       NaN  NaN  1.3125       NaN   NaN   NaN   NaN   
418    2023/02/23       NaN  NaN     NaN       NaN   NaN   NaN   NaN   
419    2023/02/24       NaN  NaN     NaN       NaN   NaN   NaN   NaN   

stock      CLOV  GOOG  ...  PL_count  PLTR_count  RIVN_count  SLA_count  \
0           NaN   NaN  ...       NaN         NaN         NaN        NaN   
1           NaN   NaN  ...       NaN         NaN         NaN        NaN   
2           NaN   NaN  ...       NaN         NaN         NaN        NaN   
3           NaN   NaN  ...       NaN         1.0         NaN        NaN   
4      0.833333   NaN  ...       NaN         NaN         1.0        NaN   
..          ...   ...  ...       ...         ...         ...        ...   
415         NaN   NaN  ...       NaN         NaN         NaN        NaN   
416         NaN   NaN  ...       NaN         NaN         NaN        NaN   
417         NaN   1.5  ...       NaN         NaN         NaN        NaN   
418         NaN   NaN  ...       NaN         NaN         1.0        NaN   
419         NaN   0.5  ...       NaN         NaN         NaN        NaN   

stock  STM_count  TSLA_count  TSM_count  TXN_count  WISH_count  ZM_count  
0            NaN         NaN        NaN        NaN         NaN       NaN  
1            NaN        40.0        NaN        NaN         NaN       NaN  
2            NaN         6.0        NaN        NaN         NaN       NaN  
3            NaN        59.0        NaN        NaN         NaN       NaN  
4            NaN       280.0        1.0        NaN         1.0       NaN  
..           ...         ...        ...        ...         ...       ...  
415          NaN         NaN        NaN        NaN         NaN       NaN  
416          NaN         NaN        NaN        NaN         NaN       NaN  
417          NaN         NaN        NaN        NaN         NaN       NaN  
418          NaN         NaN        NaN        NaN         NaN       NaN  
419          NaN         NaN        NaN        NaN         NaN       NaN  

[420 rows x 49 columns]

In [9]:
comment_time_series[['Date', 'AAPL', 'AMC', 'AMD', 'AMZN', 'AVGO', 'BABA', 'BB', 'CLOV',
       'GOOG', 'LA', 'LCID', 'META', 'MSFT', 'NIO', 'PL', 'PLTR', 'RIVN', 'SLA', 'STM', 'TSLA', 
                      'TSM', 'TXN', 'WISH', 'ZM']]=comment_time_series[['Date', 'AAPL', 'AMC', 'AMD', 'AMZN', 'AVGO', 'BABA', 'BB', 'CLOV',
       'GOOG', 'LA', 'LCID', 'META', 'MSFT', 'NIO', 'PL', 'PLTR', 'RIVN', 'SLA', 'STM', 'TSLA', 
                      'TSM', 'TXN', 'WISH', 'ZM']].fillna(method='ffill')

In [10]:
comment_time_series = comment_time_series.fillna(0)

In [11]:
comment_time_series

stock        Date      AAPL  AMC     AMD      AMZN      AVGO  BABA    BB  \
0      2021/12/31  0.000000  0.0  0.0000  0.000000  0.000000  0.00  0.00   
1      2022/01/01  0.656250  0.0  0.4375  1.364198  0.000000  1.00  0.00   
2      2022/01/02  0.750000  0.0  0.4375  1.364198  0.000000  1.00  0.00   
3      2022/01/03  0.750000  0.0  0.4375  1.364198  0.000000  1.00  0.00   
4      2022/01/04  1.222222  0.0  0.0000  0.500000  0.000000  0.50  0.75   
..            ...       ...  ...     ...       ...       ...   ...   ...   
415    2023/02/20  0.000000  0.5  1.1250  0.750000  0.555556  0.75  0.00   
416    2023/02/21  0.000000  0.5  1.1250  0.750000  0.555556  0.75  0.00   
417    2023/02/22  0.000000  0.5  1.3125  0.750000  0.555556  0.75  0.00   
418    2023/02/23  0.000000  0.5  1.3125  0.750000  0.555556  0.75  0.00   
419    2023/02/24  0.000000  0.5  1.3125  0.750000  0.555556  0.75  0.00   

stock      CLOV  GOOG  ...  PL_count  PLTR_count  RIVN_count  SLA_count  \
0      0.000000   0.0  ...       0.0         0.0         0.0        0.0   
1      0.000000   0.0  ...       0.0         0.0         0.0        0.0   
2      0.000000   0.0  ...       0.0         0.0         0.0        0.0   
3      0.000000   0.0  ...       0.0         1.0         0.0        0.0   
4      0.833333   0.0  ...       0.0         0.0         1.0        0.0   
..          ...   ...  ...       ...         ...         ...        ...   
415    0.000000   0.0  ...       0.0         0.0         0.0        0.0   
416    0.000000   0.0  ...       0.0         0.0         0.0        0.0   
417    0.000000   1.5  ...       0.0         0.0         0.0        0.0   
418    0.000000   1.5  ...       0.0         0.0         1.0        0.0   
419    0.000000   0.5  ...       0.0         0.0         0.0        0.0   

stock  STM_count  TSLA_count  TSM_count  TXN_count  WISH_count  ZM_count  
0            0.0         0.0        0.0        0.0         0.0       0.0  
1            0.0        40.0        0.0        0.0         0.0       0.0  
2            0.0         6.0        0.0        0.0         0.0       0.0  
3            0.0        59.0        0.0        0.0         0.0       0.0  
4            0.0       280.0        1.0        0.0         1.0       0.0  
..           ...         ...        ...        ...         ...       ...  
415          0.0         0.0        0.0        0.0         0.0       0.0  
416          0.0         0.0        0.0        0.0         0.0       0.0  
417          0.0         0.0        0.0        0.0         0.0       0.0  
418          0.0         0.0        0.0        0.0         0.0       0.0  
419          0.0         0.0        0.0        0.0         0.0       0.0  

[420 rows x 49 columns]

In [12]:
opinion_time_series = reformat_sentiment_indices(weighted_opinion_indices)
opinion_time_series[['Date', 'AAPL', 'AMC', 'AMD', 'AMZN', 'AVGO', 'BABA', 'BB',
       'GOOG', 'LCID', 'META', 'MSFT', 'NIO', 'PLTR', 'RIVN', 'STM', 'TSLA', 
                      'TSM', 'TXN', 'WISH', 'ZM']]=opinion_time_series[['Date', 'AAPL', 'AMC', 'AMD', 'AMZN', 'AVGO', 'BABA', 'BB', 
       'GOOG', 'LCID', 'META', 'MSFT', 'NIO', 'PLTR', 'RIVN', 'STM', 'TSLA', 
                      'TSM', 'TXN', 'WISH', 'ZM']].fillna(method='ffill')


In [13]:
opinion_time_series = opinion_time_series.fillna(0)

In [14]:
opinion_time_series

stock        Date      AAPL  AMC    AMD      AMZN  AVGO  BABA   BB      GOOG  \
0      2022/01/01  0.000000  0.0  0.000  0.277778   1.0   0.0  0.0  0.375000   
1      2022/01/02  0.000000  0.0  0.000  0.000000   1.0   0.0  0.0  0.375000   
2      2022/01/03  0.524691  0.5  0.000  0.458333   0.0   0.5  0.0  0.555556   
3      2022/01/04  0.360000  0.5  0.000  0.277778   0.0   0.5  0.0  0.555556   
4      2022/01/05  0.375000  0.5  0.000  0.500000   0.5   0.5  0.0  0.750000   
..            ...       ...  ...    ...       ...   ...   ...  ...       ...   
361    2023/02/15  0.000000  0.0  0.375  0.000000   0.5   1.0  0.0  0.500000   
362    2023/02/16  0.500000  0.0  0.375  0.000000   0.5   1.0  0.0  0.000000   
363    2023/02/19  0.500000  0.0  0.375  0.000000   0.5   1.0  0.0  0.000000   
364    2023/02/20  0.500000  0.0  0.375  0.000000   0.5   1.0  0.0  0.000000   
365    2023/02/21  0.000000  0.0  0.375  0.750000   0.5   1.0  0.0  0.500000   

stock  LCID  ...  MSFT_count  NIO_count  PLTR_count  RIVN_count  STM_count  \
0       0.0  ...         2.0        0.0         0.0         0.0        0.0   
1       0.0  ...         1.0        0.0         0.0         0.0        0.0   
2       0.0  ...         7.0        0.0         0.0         0.0        0.0   
3       0.0  ...         2.0        0.0         0.0         0.0        0.0   
4       0.0  ...         1.0        0.0         0.0         0.0        0.0   
..      ...  ...         ...        ...         ...         ...        ...   
361     0.5  ...         1.0        0.0         0.0         0.0        0.0   
362     0.5  ...         2.0        0.0         0.0         0.0        0.0   
363     0.5  ...         0.0        0.0         0.0         0.0        0.0   
364     0.5  ...         1.0        0.0         0.0         0.0        0.0   
365     0.5  ...         1.0        0.0         0.0         0.0        0.0   

stock  TSLA_count  TSM_count  TXN_count  WISH_count  ZM_count  
0             1.0        0.0        0.0         0.0       0.0  
1             1.0        0.0        0.0         0.0       0.0  
2             9.0        0.0        0.0         0.0       0.0  
3             7.0        0.0        1.0         0.0       0.0  
4             5.0        0.0        0.0         0.0       0.0  
..            ...        ...        ...         ...       ...  
361           1.0        0.0        0.0         0.0       0.0  
362           2.0        0.0        0.0         0.0       0.0  
363           0.0        0.0        0.0         0.0       0.0  
364           0.0        0.0        0.0         0.0       0.0  
365           0.0        0.0        0.0         0.0       0.0  

[366 rows x 41 columns]

### 分离每个股票

#### tesla

In [17]:
tesla_comment_sentiment = comment_time_series[['Date','TSLA','TSLA_count']]
tesla_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']

In [18]:
tesla_opinion_sentiment = opinion_time_series[['Date','TSLA','TSLA_count']]
tesla_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']

In [19]:
testla_sentiment = pd.merge(tesla_comment_sentiment, tesla_opinion_sentiment, on='Date', how='left')
testla_sentiment.dropna()

Date  comment_sentiment  comment_count  opinion_sentiment  \
1    2022/01/01           1.061562           40.0           0.000000   
2    2022/01/02           0.763889            6.0           0.000000   
3    2022/01/03           1.058747           59.0           0.419753   
4    2022/01/04           1.062385          280.0           0.265306   
5    2022/01/05           0.902937          118.0           0.360000   
..          ...                ...            ...                ...   
410  2023/02/14           0.524691            9.0           1.125000   
411  2023/02/15           0.918210           18.0           0.500000   
412  2023/02/16           0.918210            0.0           0.000000   
415  2023/02/20           0.918210            0.0           0.000000   
416  2023/02/21           0.918210            0.0           0.000000   

     opinion_count  
1              1.0  
2              1.0  
3              9.0  
4              7.0  
5              5.0  
..             ...  
410            0.0  
411            1.0  
412            2.0  
415            0.0  
416            0.0  

[365 rows x 5 columns]

In [20]:
tsla_price = pd.read_csv('TSLA Historical Data.csv')

In [24]:
tsla_price['Date'] = pd.to_datetime(tsla_price['Date'], format='%m/%d/%Y')
testla_sentiment['Date'] = pd.to_datetime(testla_sentiment['Date'])
merged_df = pd.merge(tsla_price, testla_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
print(merged_df.head())

        Date   Price    Open    High     Low     Vol. Change %  \
0 2023-02-24  196.88  196.32  197.67  192.80  142.23M   -2.57%   
1 2023-02-23  202.07  203.91  205.14  196.33  146.36M    0.60%   
2 2023-02-22  200.86  197.93  201.99  191.78  191.83M    1.77%   
3 2023-02-21  197.37  204.99  209.71  197.22  180.02M   -5.25%   
4 2023-02-17  208.31  199.99  208.44  197.50  213.74M    3.10%   

   comment_sentiment  comment_count  opinion_sentiment  opinion_count  
0            0.91821            0.0                0.0            0.0  
1            0.91821            0.0                0.0            0.0  
2            0.91821            0.0                0.0            0.0  
3            0.91821            0.0                0.0            0.0  
4            0.91821            0.0                0.0            0.0  


In [25]:
TSLA_merged_df = merged_df.iloc[::-1]
TSLA_merged_df.head()

Date   Price    Open    High     Low     Vol. Change %  \
288 2021-12-31  352.26  357.81  360.67  351.53   40.73M   -1.27%   
287 2022-01-03  399.93  382.58  400.36  378.68  104.69M   13.53%   
286 2022-01-04  383.20  396.52  402.67  374.35  100.25M   -4.18%   
285 2022-01-05  362.71  382.22  390.11  360.34   80.12M   -5.35%   
284 2022-01-06  354.90  359.00  362.67  340.17   90.34M   -2.15%   

     comment_sentiment  comment_count  opinion_sentiment  opinion_count  
288           0.000000            0.0           0.000000            0.0  
287           1.058747           59.0           0.419753            9.0  
286           1.062385          280.0           0.265306            7.0  
285           0.902937          118.0           0.360000            5.0  
284           1.021079          118.0           0.360000            0.0

In [26]:
TSLA_merged_df['comment_sentiment_change'] = TSLA_merged_df['comment_sentiment'] - TSLA_merged_df['comment_sentiment'].shift(1)
TSLA_merged_df['opinion_sentiment_change'] = TSLA_merged_df['opinion_sentiment'] - TSLA_merged_df['opinion_sentiment'].shift(1)
TSLA_merged_df.head()

Date   Price    Open    High     Low     Vol. Change %  \
288 2021-12-31  352.26  357.81  360.67  351.53   40.73M   -1.27%   
287 2022-01-03  399.93  382.58  400.36  378.68  104.69M   13.53%   
286 2022-01-04  383.20  396.52  402.67  374.35  100.25M   -4.18%   
285 2022-01-05  362.71  382.22  390.11  360.34   80.12M   -5.35%   
284 2022-01-06  354.90  359.00  362.67  340.17   90.34M   -2.15%   

     comment_sentiment  comment_count  opinion_sentiment  opinion_count  \
288           0.000000            0.0           0.000000            0.0   
287           1.058747           59.0           0.419753            9.0   
286           1.062385          280.0           0.265306            7.0   
285           0.902937          118.0           0.360000            5.0   
284           1.021079          118.0           0.360000            0.0   

     comment_sentiment_change  opinion_sentiment_change  
288                       NaN                       NaN  
287                  1.058747                  0.419753  
286                  0.003638                 -0.154447  
285                 -0.159448                  0.094694  
284                  0.118141                  0.000000

In [27]:
TSLA_merged_df['lag_-1_comment_sentiment'] = TSLA_merged_df['comment_sentiment'].shift(1) #前一天的
TSLA_merged_df['lag_-2_comment_sentiment'] = TSLA_merged_df['comment_sentiment'].shift(2)
TSLA_merged_df['lag_1_comment_sentiment'] = TSLA_merged_df['comment_sentiment'].shift(-1)
TSLA_merged_df['lag_2_comment_sentiment'] = TSLA_merged_df['comment_sentiment'].shift(-2)
TSLA_merged_df['lag_-1_opinion_sentiment'] = TSLA_merged_df['opinion_sentiment'].shift(1) #前一天的
TSLA_merged_df['lag_-2_opinion_sentiment'] = TSLA_merged_df['opinion_sentiment'].shift(2)
TSLA_merged_df['lag_1_opinion_sentiment'] = TSLA_merged_df['opinion_sentiment'].shift(-1)
TSLA_merged_df['lag_2_opinion_sentiment'] = TSLA_merged_df['opinion_sentiment'].shift(-2)

TSLA_merged_df['lag_-1_comment_count'] = TSLA_merged_df['comment_count'].shift(1)
TSLA_merged_df['lag_1_comment_count'] = TSLA_merged_df['comment_count'].shift(-1)
TSLA_merged_df['lag_-1_opinion_count'] = TSLA_merged_df['opinion_count'].shift(1)
TSLA_merged_df['lag_1_opinion_count'] = TSLA_merged_df['opinion_count'].shift(-1)


TSLA_merged_df['Change %'] = TSLA_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
TSLA_merged_df['lag_1_change'] = TSLA_merged_df['Change %'].shift(1)
TSLA_merged_df['direction'] = (TSLA_merged_df['Change %'] > 0).astype(int)
TSLA_merged_df['Vol.'] = TSLA_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
TSLA_merged_df['lag_1_Vol.'] = TSLA_merged_df['Vol.'].shift(1)
# TSLA_merged_df['lag_1_sentiment_change'] = TSLA_merged_df['sentiment_change'].shift(1)
TSLA_merged_df = TSLA_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
TSLA_merged_df = TSLA_merged_df.rename(columns={'comment_count': '0_comment_count'})
TSLA_merged_df = TSLA_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
TSLA_merged_df = TSLA_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
TSLA_merged_df = TSLA_merged_df.dropna()
TSLA_merged_df.head()

Date   Price    Open    High     Low    Vol.  Change %  \
286 2022-01-04  383.20  396.52  402.67  374.35  100.25     -4.18   
285 2022-01-05  362.71  382.22  390.11  360.34   80.12     -5.35   
284 2022-01-06  354.90  359.00  362.67  340.17   90.34     -2.15   
283 2022-01-07  342.32  360.12  360.31  336.67   84.17     -3.54   
282 2022-01-10  352.71  333.33  353.03  326.67   91.82      3.03   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             1.062385            280.0             0.265306  ...   
285             0.902937            118.0             0.360000  ...   
284             1.021079            118.0             0.360000  ...   
283             1.027593             90.0             0.750000  ...   
282             0.585938              8.0             0.437500  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                  0.000000                   0.3600   
285                  0.419753                   0.3600   
284                  0.265306                   0.7500   
283                  0.360000                   0.4375   
282                  0.360000                   0.8750   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                   0.3600                  59.0                118.0   
285                   0.7500                 280.0                118.0   
284                   0.4375                 118.0                 90.0   
283                   0.8750                 118.0                  8.0   
282                   0.7500                  90.0                150.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   9.0                  5.0         13.53          0   
285                   7.0                  0.0         -4.18          0   
284                   5.0                  2.0         -5.35          0   
283                   0.0                  4.0         -2.15          0   
282                   2.0                  4.0         -3.54          1   

     lag_1_Vol.  
286      104.69  
285      100.25  
284       80.12  
283       90.34  
282       84.17  

[5 rows x 28 columns]

In [28]:
TSLA_merged_df.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %',
       '0_comment_sentiment', '0_comment_count', '0_opinion_sentiment',
       '0_opinion_count', 'comment_sentiment_change',
       'opinion_sentiment_change', 'lag_-1_comment_sentiment',
       'lag_-2_comment_sentiment', 'lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-1_opinion_sentiment',
       'lag_-2_opinion_sentiment', 'lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment', 'lag_-1_comment_count',
       'lag_1_comment_count', 'lag_-1_opinion_count', 'lag_1_opinion_count',
       'lag_1_change', 'direction', 'lag_1_Vol.'],
      dtype='object')

##### 验证sentiment具备prediction power

In [30]:
import statsmodels.api as sm
X = TSLA_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = TSLA_merged_df['direction']
model_tesla = sm.OLS(y, X).fit()

# Print the model summary
print(model_tesla.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.526
Model:                            OLS   Adj. R-squared (uncentered):              0.498
Method:                 Least Squares   F-statistic:                              18.67
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    5.37e-35
Time:                        09:35:24   Log-Likelihood:                         -201.68
No. Observations:                 285   AIC:                                      435.4
Df Residuals:                     269   BIC:                                      493.8
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### NIO

In [31]:
NIO_comment_sentiment = comment_time_series[['Date','NIO','NIO_count']]
NIO_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
NIO_opinion_sentiment = opinion_time_series[['Date','NIO','NIO_count']]
NIO_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
NIO_sentiment = pd.merge(NIO_comment_sentiment, NIO_opinion_sentiment, on='Date', how='left')
NIO_sentiment.dropna()
NIO_price = pd.read_csv('NIO Historical Data.csv')
NIO_price['Date'] = pd.to_datetime(NIO_price['Date'], format='%m/%d/%Y')
NIO_sentiment['Date'] = pd.to_datetime(NIO_sentiment['Date'])
merged_df = pd.merge(NIO_price, NIO_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
NIO_merged_df = merged_df.iloc[::-1]
NIO_merged_df['comment_sentiment_change'] = NIO_merged_df['comment_sentiment'] - NIO_merged_df['comment_sentiment'].shift(1)
NIO_merged_df['opinion_sentiment_change'] = NIO_merged_df['opinion_sentiment'] - NIO_merged_df['opinion_sentiment'].shift(1)
NIO_merged_df['lag_-1_comment_sentiment'] = NIO_merged_df['comment_sentiment'].shift(1) #前一天的
NIO_merged_df['lag_-2_comment_sentiment'] = NIO_merged_df['comment_sentiment'].shift(2)
NIO_merged_df['lag_1_comment_sentiment'] = NIO_merged_df['comment_sentiment'].shift(-1)
NIO_merged_df['lag_2_comment_sentiment'] = NIO_merged_df['comment_sentiment'].shift(-2)
NIO_merged_df['lag_-1_opinion_sentiment'] = NIO_merged_df['opinion_sentiment'].shift(1) #前一天的
NIO_merged_df['lag_-2_opinion_sentiment'] = NIO_merged_df['opinion_sentiment'].shift(2)
NIO_merged_df['lag_1_opinion_sentiment'] = NIO_merged_df['opinion_sentiment'].shift(-1)
NIO_merged_df['lag_2_opinion_sentiment'] = NIO_merged_df['opinion_sentiment'].shift(-2)

NIO_merged_df['lag_-1_comment_count'] = NIO_merged_df['comment_count'].shift(1)
NIO_merged_df['lag_1_comment_count'] = NIO_merged_df['comment_count'].shift(-1)
NIO_merged_df['lag_-1_opinion_count'] = NIO_merged_df['opinion_count'].shift(1)
NIO_merged_df['lag_1_opinion_count'] = NIO_merged_df['opinion_count'].shift(-1)

NIO_merged_df['Change %'] = NIO_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
NIO_merged_df['lag_1_change'] = NIO_merged_df['Change %'].shift(1)
NIO_merged_df['direction'] = (NIO_merged_df['Change %'] > 0).astype(int)
NIO_merged_df['Vol.'] = NIO_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
NIO_merged_df['lag_1_Vol.'] = NIO_merged_df['Vol.'].shift(1)
NIO_merged_df = NIO_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
NIO_merged_df = NIO_merged_df.rename(columns={'comment_count': '0_comment_count'})
NIO_merged_df = NIO_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
NIO_merged_df = NIO_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
NIO_merged_df = NIO_merged_df.dropna()
NIO_merged_df.head()

Date  Price   Open   High    Low   Vol.  Change %  \
286 2022-01-04  31.58  33.35  33.39  31.11  47.87     -5.65   
285 2022-01-05  29.88  30.97  31.94  29.78  42.44     -5.38   
284 2022-01-06  29.31  30.01  30.54  28.28  48.72     -1.91   
283 2022-01-07  29.30  29.45  30.42  28.78  40.78     -0.03   
282 2022-01-10  28.91  29.00  29.16  27.78  44.83     -1.33   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             0.938368             24.0                  0.0  ...   
285             0.948870             29.0                  0.0  ...   
284             0.971876             53.0                  0.0  ...   
283             1.035990             41.0                  0.0  ...   
282             0.928571              7.0                  0.0  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                       0.0                      0.0   
285                       0.0                      0.0   
284                       0.0                      0.0   
283                       0.0                      0.0   
282                       0.0                      1.0   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                      0.0                   9.0                 29.0   
285                      0.0                  24.0                 53.0   
284                      0.0                  29.0                 41.0   
283                      1.0                  53.0                  7.0   
282                      1.0                  41.0                 23.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   0.0                  0.0          5.65          0   
285                   0.0                  0.0         -5.65          0   
284                   0.0                  0.0         -5.38          0   
283                   0.0                  0.0         -1.91          0   
282                   0.0                  1.0         -0.03          0   

     lag_1_Vol.  
286       64.41  
285       47.87  
284       42.44  
283       48.72  
282       40.78  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [32]:
import statsmodels.api as sm
X = NIO_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = NIO_merged_df['direction']
model_nio = sm.OLS(y, X).fit()

# Print the model summary
print(model_nio.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.435
Model:                            OLS   Adj. R-squared (uncentered):              0.401
Method:                 Least Squares   F-statistic:                              12.93
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    2.89e-25
Time:                        09:35:43   Log-Likelihood:                         -204.51
No. Observations:                 285   AIC:                                      441.0
Df Residuals:                     269   BIC:                                      499.5
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### BABA

In [33]:
BABA_comment_sentiment = comment_time_series[['Date','BABA','BABA_count']]
BABA_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
BABA_opinion_sentiment = opinion_time_series[['Date','BABA','BABA_count']]
BABA_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
BABA_sentiment = pd.merge(BABA_comment_sentiment, BABA_opinion_sentiment, on='Date', how='left')
BABA_sentiment.dropna()
BABA_price = pd.read_csv('BABA Historical Data.csv')
BABA_price['Date'] = pd.to_datetime(BABA_price['Date'], format='%m/%d/%Y')
BABA_sentiment['Date'] = pd.to_datetime(BABA_sentiment['Date'])
merged_df = pd.merge(BABA_price, BABA_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
BABA_merged_df = merged_df.iloc[::-1]
BABA_merged_df['comment_sentiment_change'] = BABA_merged_df['comment_sentiment'] - BABA_merged_df['comment_sentiment'].shift(1)
BABA_merged_df['opinion_sentiment_change'] = BABA_merged_df['opinion_sentiment'] - BABA_merged_df['opinion_sentiment'].shift(1)
BABA_merged_df['lag_-1_comment_sentiment'] = BABA_merged_df['comment_sentiment'].shift(1) #前一天的
BABA_merged_df['lag_-2_comment_sentiment'] = BABA_merged_df['comment_sentiment'].shift(2)
BABA_merged_df['lag_1_comment_sentiment'] = BABA_merged_df['comment_sentiment'].shift(-1)
BABA_merged_df['lag_2_comment_sentiment'] = BABA_merged_df['comment_sentiment'].shift(-2)
BABA_merged_df['lag_-1_opinion_sentiment'] = BABA_merged_df['opinion_sentiment'].shift(1) #前一天的
BABA_merged_df['lag_-2_opinion_sentiment'] = BABA_merged_df['opinion_sentiment'].shift(2)
BABA_merged_df['lag_1_opinion_sentiment'] = BABA_merged_df['opinion_sentiment'].shift(-1)
BABA_merged_df['lag_2_opinion_sentiment'] = BABA_merged_df['opinion_sentiment'].shift(-2)

BABA_merged_df['lag_-1_comment_count'] = BABA_merged_df['comment_count'].shift(1)
BABA_merged_df['lag_1_comment_count'] = BABA_merged_df['comment_count'].shift(-1)
BABA_merged_df['lag_-1_opinion_count'] = BABA_merged_df['opinion_count'].shift(1)
BABA_merged_df['lag_1_opinion_count'] = BABA_merged_df['opinion_count'].shift(-1)

BABA_merged_df['Change %'] = BABA_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
BABA_merged_df['lag_1_change'] = BABA_merged_df['Change %'].shift(1)
BABA_merged_df['direction'] = (BABA_merged_df['Change %'] > 0).astype(int)
BABA_merged_df['Vol.'] = BABA_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
BABA_merged_df['lag_1_Vol.'] = BABA_merged_df['Vol.'].shift(1)
BABA_merged_df = BABA_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
BABA_merged_df = BABA_merged_df.rename(columns={'comment_count': '0_comment_count'})
BABA_merged_df = BABA_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
BABA_merged_df = BABA_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
BABA_merged_df = BABA_merged_df.dropna()
BABA_merged_df.head()

Date   Price    Open    High     Low   Vol.  Change %  \
286 2022-01-04  119.56  119.53  120.87  115.77  20.03     -0.68   
285 2022-01-05  121.16  118.00  126.62  117.70  35.24      1.34   
284 2022-01-06  126.63  124.26  128.40  123.46  31.12      4.51   
283 2022-01-07  129.81  130.24  133.88  128.18  38.11      2.51   
282 2022-01-10  128.30  131.99  132.60  126.23  21.74     -1.16   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             0.500000              1.0                  0.5  ...   
285             1.154321              9.0                  0.5  ...   
284             1.035503             13.0                  0.5  ...   
283             1.069444              6.0                  0.5  ...   
282             0.944444              0.0                  1.0  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                       0.0                      0.5   
285                       0.5                      0.5   
284                       0.5                      0.5   
283                       0.5                      1.0   
282                       0.5                      1.0   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                      0.5                   0.0                  9.0   
285                      0.5                   1.0                 13.0   
284                      1.0                   9.0                  6.0   
283                      1.0                  13.0                  0.0   
282                      1.0                   6.0                  5.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   1.0                  0.0          1.34          0   
285                   0.0                  0.0         -0.68          1   
284                   0.0                  1.0          1.34          1   
283                   0.0                  1.0          4.51          1   
282                   1.0                  0.0          2.51          0   

     lag_1_Vol.  
286       16.44  
285       20.03  
284       35.24  
283       31.12  
282       38.11  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [34]:
import statsmodels.api as sm
X = BABA_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = BABA_merged_df['direction']
model_baba = sm.OLS(y, X).fit()

# Print the model summary
print(model_baba.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.410
Model:                            OLS   Adj. R-squared (uncentered):              0.375
Method:                 Least Squares   F-statistic:                              11.70
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    5.77e-23
Time:                        09:35:57   Log-Likelihood:                         -211.69
No. Observations:                 285   AIC:                                      455.4
Df Residuals:                     269   BIC:                                      513.8
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### AMD

In [35]:
AMD_comment_sentiment = comment_time_series[['Date','AMD','AMD_count']]
AMD_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
AMD_opinion_sentiment = opinion_time_series[['Date','AMD','AMD_count']]
AMD_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
AMD_sentiment = pd.merge(AMD_comment_sentiment, AMD_opinion_sentiment, on='Date', how='left')
AMD_sentiment.dropna()
AMD_price = pd.read_csv('AMD Historical Data.csv')
AMD_price['Date'] = pd.to_datetime(AMD_price['Date'], format='%m/%d/%Y')
AMD_sentiment['Date'] = pd.to_datetime(AMD_sentiment['Date'])
merged_df = pd.merge(AMD_price, AMD_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
AMD_merged_df = merged_df.iloc[::-1]
AMD_merged_df['comment_sentiment_change'] = AMD_merged_df['comment_sentiment'] - AMD_merged_df['comment_sentiment'].shift(1)
AMD_merged_df['opinion_sentiment_change'] = AMD_merged_df['opinion_sentiment'] - AMD_merged_df['opinion_sentiment'].shift(1)
AMD_merged_df['lag_-1_comment_sentiment'] = AMD_merged_df['comment_sentiment'].shift(1) #前一天的
AMD_merged_df['lag_-2_comment_sentiment'] = AMD_merged_df['comment_sentiment'].shift(2)
AMD_merged_df['lag_1_comment_sentiment'] = AMD_merged_df['comment_sentiment'].shift(-1)
AMD_merged_df['lag_2_comment_sentiment'] = AMD_merged_df['comment_sentiment'].shift(-2)
AMD_merged_df['lag_-1_opinion_sentiment'] = AMD_merged_df['opinion_sentiment'].shift(1) #前一天的
AMD_merged_df['lag_-2_opinion_sentiment'] = AMD_merged_df['opinion_sentiment'].shift(2)
AMD_merged_df['lag_1_opinion_sentiment'] = AMD_merged_df['opinion_sentiment'].shift(-1)
AMD_merged_df['lag_2_opinion_sentiment'] = AMD_merged_df['opinion_sentiment'].shift(-2)

AMD_merged_df['lag_-1_comment_count'] = AMD_merged_df['comment_count'].shift(1)
AMD_merged_df['lag_1_comment_count'] = AMD_merged_df['comment_count'].shift(-1)
AMD_merged_df['lag_-1_opinion_count'] = AMD_merged_df['opinion_count'].shift(1)
AMD_merged_df['lag_1_opinion_count'] = AMD_merged_df['opinion_count'].shift(-1)

AMD_merged_df['Change %'] = AMD_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
AMD_merged_df['lag_1_change'] = AMD_merged_df['Change %'].shift(1)
AMD_merged_df['direction'] = (AMD_merged_df['Change %'] > 0).astype(int)
AMD_merged_df['Vol.'] = AMD_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
AMD_merged_df['lag_1_Vol.'] = AMD_merged_df['Vol.'].shift(1)
AMD_merged_df = AMD_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
AMD_merged_df = AMD_merged_df.rename(columns={'comment_count': '0_comment_count'})
AMD_merged_df = AMD_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
AMD_merged_df = AMD_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
AMD_merged_df = AMD_merged_df.dropna()
AMD_merged_df.head()

Date   Price    Open    High     Low   Vol.  Change %  \
286 2022-01-04  144.42  151.01  152.42  140.70  80.20     -3.87   
285 2022-01-05  136.15  142.82  143.76  135.29  65.40     -5.73   
284 2022-01-06  136.23  136.19  138.00  131.77  64.80      0.06   
283 2022-01-07  132.00  136.28  137.44  131.13  58.44     -3.11   
282 2022-01-10  132.00  129.08  132.42  125.03  84.67      0.00   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             0.000000              1.0                 0.00  ...   
285             0.375000              2.0                 0.00  ...   
284             0.000000              2.0                 0.00  ...   
283             0.360000              5.0                 0.75  ...   
282             0.833333              0.0                 0.50  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                       0.0                     0.00   
285                       0.0                     0.00   
284                       0.0                     0.75   
283                       0.0                     0.50   
282                       0.0                     0.00   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                     0.00                   0.0                  2.0   
285                     0.75                   1.0                  2.0   
284                     0.50                   2.0                  5.0   
283                     0.00                   2.0                  0.0   
282                     0.50                   5.0                  4.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   1.0                  0.0          4.41          0   
285                   2.0                  1.0         -3.87          0   
284                   0.0                  2.0         -5.73          1   
283                   1.0                  1.0          0.06          0   
282                   2.0                  1.0         -3.11          0   

     lag_1_Vol.  
286       59.48  
285       80.20  
284       65.40  
283       64.80  
282       58.44  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [36]:
import statsmodels.api as sm
X = AMD_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = AMD_merged_df['direction']
model_amd = sm.OLS(y, X).fit()

# Print the model summary
print(model_amd.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.442
Model:                            OLS   Adj. R-squared (uncentered):              0.409
Method:                 Least Squares   F-statistic:                              13.31
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    5.87e-26
Time:                        09:36:07   Log-Likelihood:                         -210.53
No. Observations:                 285   AIC:                                      453.1
Df Residuals:                     269   BIC:                                      511.5
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### META

In [37]:
META_comment_sentiment = comment_time_series[['Date','META','META_count']]
META_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
META_opinion_sentiment = opinion_time_series[['Date','META','META_count']]
META_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
META_sentiment = pd.merge(META_comment_sentiment, META_opinion_sentiment, on='Date', how='left')
META_sentiment.dropna()
META_price = pd.read_csv('META Historical Data.csv')
META_price['Date'] = pd.to_datetime(META_price['Date'], format='%m/%d/%Y')
META_sentiment['Date'] = pd.to_datetime(META_sentiment['Date'])
merged_df = pd.merge(META_price, META_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
META_merged_df = merged_df.iloc[::-1]
META_merged_df['comment_sentiment_change'] = META_merged_df['comment_sentiment'] - META_merged_df['comment_sentiment'].shift(1)
META_merged_df['opinion_sentiment_change'] = META_merged_df['opinion_sentiment'] - META_merged_df['opinion_sentiment'].shift(1)
META_merged_df['lag_-1_comment_sentiment'] = META_merged_df['comment_sentiment'].shift(1) #前一天的
META_merged_df['lag_-2_comment_sentiment'] = META_merged_df['comment_sentiment'].shift(2)
META_merged_df['lag_1_comment_sentiment'] = META_merged_df['comment_sentiment'].shift(-1)
META_merged_df['lag_2_comment_sentiment'] = META_merged_df['comment_sentiment'].shift(-2)
META_merged_df['lag_-1_opinion_sentiment'] = META_merged_df['opinion_sentiment'].shift(1) #前一天的
META_merged_df['lag_-2_opinion_sentiment'] = META_merged_df['opinion_sentiment'].shift(2)
META_merged_df['lag_1_opinion_sentiment'] = META_merged_df['opinion_sentiment'].shift(-1)
META_merged_df['lag_2_opinion_sentiment'] = META_merged_df['opinion_sentiment'].shift(-2)

META_merged_df['lag_-1_comment_count'] = META_merged_df['comment_count'].shift(1)
META_merged_df['lag_1_comment_count'] = META_merged_df['comment_count'].shift(-1)
META_merged_df['lag_-1_opinion_count'] = META_merged_df['opinion_count'].shift(1)
META_merged_df['lag_1_opinion_count'] = META_merged_df['opinion_count'].shift(-1)

META_merged_df['Change %'] = META_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
META_merged_df['lag_1_change'] = META_merged_df['Change %'].shift(1)
META_merged_df['direction'] = (META_merged_df['Change %'] > 0).astype(int)
META_merged_df['Vol.'] = META_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
META_merged_df['lag_1_Vol.'] = META_merged_df['Vol.'].shift(1)
META_merged_df = META_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
META_merged_df = META_merged_df.rename(columns={'comment_count': '0_comment_count'})
META_merged_df = META_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
META_merged_df = META_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
META_merged_df = META_merged_df.dropna()
META_merged_df.head()

Date   Price    Open    High     Low   Vol.  Change %  \
286 2022-01-04  336.53  339.95  343.09  331.87  16.00     -0.59   
285 2022-01-05  324.17  333.02  335.76  323.84  20.56     -3.67   
284 2022-01-06  332.46  322.82  339.17  322.72  27.96      2.56   
283 2022-01-07  331.79  332.74  337.00  328.88  14.72     -0.20   
282 2022-01-10  328.07  325.31  328.34  315.43  24.94     -1.12   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286                  0.0              1.0                 0.75  ...   
285                  1.0              1.0                 0.75  ...   
284                  1.0              0.0                 0.75  ...   
283                  0.0              1.0                 0.75  ...   
282                  0.0              1.0                 0.00  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                      0.00                     0.75   
285                      0.00                     0.75   
284                      0.75                     0.75   
283                      0.75                     0.00   
282                      0.75                     0.00   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                     0.75                   0.0                  1.0   
285                     0.75                   1.0                  0.0   
284                     0.00                   1.0                  1.0   
283                     0.00                   0.0                  1.0   
282                     0.00                   1.0                  2.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   0.0                  0.0          0.65          0   
285                   2.0                  0.0         -0.59          0   
284                   0.0                  0.0         -3.67          1   
283                   0.0                  1.0          2.56          0   
282                   0.0                  0.0         -0.20          0   

     lag_1_Vol.  
286       14.56  
285       16.00  
284       20.56  
283       27.96  
282       14.72  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [38]:
import statsmodels.api as sm
X = META_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = META_merged_df['direction']
model_meta = sm.OLS(y, X).fit()

# Print the model summary
print(model_meta.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.456
Model:                            OLS   Adj. R-squared (uncentered):              0.424
Method:                 Least Squares   F-statistic:                              14.10
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    2.22e-27
Time:                        09:36:18   Log-Likelihood:                         -212.16
No. Observations:                 285   AIC:                                      456.3
Df Residuals:                     269   BIC:                                      514.8
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### LCID

In [39]:
LCID_comment_sentiment = comment_time_series[['Date','LCID','LCID_count']]
LCID_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
LCID_opinion_sentiment = opinion_time_series[['Date','LCID','LCID_count']]
LCID_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
LCID_sentiment = pd.merge(LCID_comment_sentiment, LCID_opinion_sentiment, on='Date', how='left')
LCID_sentiment.dropna()
LCID_price = pd.read_csv('LCID Historical Data.csv')
LCID_price['Date'] = pd.to_datetime(LCID_price['Date'], format='%m/%d/%Y')
LCID_sentiment['Date'] = pd.to_datetime(LCID_sentiment['Date'])
merged_df = pd.merge(LCID_price, LCID_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
LCID_merged_df = merged_df.iloc[::-1]
LCID_merged_df['comment_sentiment_change'] = LCID_merged_df['comment_sentiment'] - LCID_merged_df['comment_sentiment'].shift(1)
LCID_merged_df['opinion_sentiment_change'] = LCID_merged_df['opinion_sentiment'] - LCID_merged_df['opinion_sentiment'].shift(1)
LCID_merged_df['lag_-1_comment_sentiment'] = LCID_merged_df['comment_sentiment'].shift(1) #前一天的
LCID_merged_df['lag_-2_comment_sentiment'] = LCID_merged_df['comment_sentiment'].shift(2)
LCID_merged_df['lag_1_comment_sentiment'] = LCID_merged_df['comment_sentiment'].shift(-1)
LCID_merged_df['lag_2_comment_sentiment'] = LCID_merged_df['comment_sentiment'].shift(-2)
LCID_merged_df['lag_-1_opinion_sentiment'] = LCID_merged_df['opinion_sentiment'].shift(1) #前一天的
LCID_merged_df['lag_-2_opinion_sentiment'] = LCID_merged_df['opinion_sentiment'].shift(2)
LCID_merged_df['lag_1_opinion_sentiment'] = LCID_merged_df['opinion_sentiment'].shift(-1)
LCID_merged_df['lag_2_opinion_sentiment'] = LCID_merged_df['opinion_sentiment'].shift(-2)

LCID_merged_df['lag_-1_comment_count'] = LCID_merged_df['comment_count'].shift(1)
LCID_merged_df['lag_1_comment_count'] = LCID_merged_df['comment_count'].shift(-1)
LCID_merged_df['lag_-1_opinion_count'] = LCID_merged_df['opinion_count'].shift(1)
LCID_merged_df['lag_1_opinion_count'] = LCID_merged_df['opinion_count'].shift(-1)

LCID_merged_df['Change %'] = LCID_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
LCID_merged_df['lag_1_change'] = LCID_merged_df['Change %'].shift(1)
LCID_merged_df['direction'] = (LCID_merged_df['Change %'] > 0).astype(int)
LCID_merged_df['Vol.'] = LCID_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
LCID_merged_df['lag_1_Vol.'] = LCID_merged_df['Vol.'].shift(1)
LCID_merged_df = LCID_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
LCID_merged_df = LCID_merged_df.rename(columns={'comment_count': '0_comment_count'})
LCID_merged_df = LCID_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
LCID_merged_df = LCID_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
LCID_merged_df = LCID_merged_df.dropna()
LCID_merged_df.head()

Date  Price   Open   High    Low   Vol.  Change %  \
286 2022-01-04  39.41  40.75  41.29  38.38  30.48     -3.71   
285 2022-01-05  36.68  38.68  39.06  36.13  43.32     -6.93   
284 2022-01-06  38.22  36.45  38.58  33.76  58.38      4.20   
283 2022-01-07  41.98  37.51  43.29  37.50  78.42      9.84   
282 2022-01-10  41.72  40.63  43.17  40.43  44.35     -0.62   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286                0.000              6.0                  0.0  ...   
285                0.000              1.0                  0.0  ...   
284                0.360              5.0                  0.0  ...   
283                0.375              2.0                  0.0  ...   
282                0.720              0.0                  0.0  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                       0.0                      0.0   
285                       0.0                      0.0   
284                       0.0                      0.0   
283                       0.0                      0.0   
282                       0.0                      0.5   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                      0.0                   0.0                  1.0   
285                      0.0                   6.0                  5.0   
284                      0.0                   1.0                  2.0   
283                      0.5                   5.0                  0.0   
282                      0.5                   2.0                  1.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   0.0                  0.0          7.57          0   
285                   0.0                  0.0         -3.71          0   
284                   0.0                  0.0         -6.93          1   
283                   0.0                  0.0          4.20          1   
282                   0.0                  1.0          9.84          0   

     lag_1_Vol.  
286       45.76  
285       30.48  
284       43.32  
283       58.38  
282       78.42  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [40]:
import statsmodels.api as sm
X = LCID_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = LCID_merged_df['direction']
model_lcid = sm.OLS(y, X).fit()

# Print the model summary
print(model_lcid.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.373
Model:                            OLS   Adj. R-squared (uncentered):              0.335
Method:                 Least Squares   F-statistic:                              9.984
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    1.25e-19
Time:                        09:36:29   Log-Likelihood:                         -228.29
No. Observations:                 285   AIC:                                      488.6
Df Residuals:                     269   BIC:                                      547.0
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### AMZN

In [41]:
AMZN_comment_sentiment = comment_time_series[['Date','AMZN','AMZN_count']]
AMZN_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
AMZN_opinion_sentiment = opinion_time_series[['Date','AMZN','AMZN_count']]
AMZN_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
AMZN_sentiment = pd.merge(AMZN_comment_sentiment, AMZN_opinion_sentiment, on='Date', how='left')
AMZN_sentiment.dropna()
AMZN_price = pd.read_csv('AMZN Historical Data.csv')
AMZN_price['Date'] = pd.to_datetime(AMZN_price['Date'], format='%m/%d/%Y')
AMZN_sentiment['Date'] = pd.to_datetime(AMZN_sentiment['Date'])
merged_df = pd.merge(AMZN_price, AMZN_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
AMZN_merged_df = merged_df.iloc[::-1]
AMZN_merged_df['comment_sentiment_change'] = AMZN_merged_df['comment_sentiment'] - AMZN_merged_df['comment_sentiment'].shift(1)
AMZN_merged_df['opinion_sentiment_change'] = AMZN_merged_df['opinion_sentiment'] - AMZN_merged_df['opinion_sentiment'].shift(1)
AMZN_merged_df['lag_-1_comment_sentiment'] = AMZN_merged_df['comment_sentiment'].shift(1) #前一天的
AMZN_merged_df['lag_-2_comment_sentiment'] = AMZN_merged_df['comment_sentiment'].shift(2)
AMZN_merged_df['lag_1_comment_sentiment'] = AMZN_merged_df['comment_sentiment'].shift(-1)
AMZN_merged_df['lag_2_comment_sentiment'] = AMZN_merged_df['comment_sentiment'].shift(-2)
AMZN_merged_df['lag_-1_opinion_sentiment'] = AMZN_merged_df['opinion_sentiment'].shift(1) #前一天的
AMZN_merged_df['lag_-2_opinion_sentiment'] = AMZN_merged_df['opinion_sentiment'].shift(2)
AMZN_merged_df['lag_1_opinion_sentiment'] = AMZN_merged_df['opinion_sentiment'].shift(-1)
AMZN_merged_df['lag_2_opinion_sentiment'] = AMZN_merged_df['opinion_sentiment'].shift(-2)

AMZN_merged_df['lag_-1_comment_count'] = AMZN_merged_df['comment_count'].shift(1)
AMZN_merged_df['lag_1_comment_count'] = AMZN_merged_df['comment_count'].shift(-1)
AMZN_merged_df['lag_-1_opinion_count'] = AMZN_merged_df['opinion_count'].shift(1)
AMZN_merged_df['lag_1_opinion_count'] = AMZN_merged_df['opinion_count'].shift(-1)

AMZN_merged_df['Change %'] = AMZN_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
AMZN_merged_df['lag_1_change'] = AMZN_merged_df['Change %'].shift(1)
AMZN_merged_df['direction'] = (AMZN_merged_df['Change %'] > 0).astype(int)
AMZN_merged_df['Vol.'] = AMZN_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
AMZN_merged_df['lag_1_Vol.'] = AMZN_merged_df['Vol.'].shift(1)
AMZN_merged_df = AMZN_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
AMZN_merged_df = AMZN_merged_df.rename(columns={'comment_count': '0_comment_count'})
AMZN_merged_df = AMZN_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
AMZN_merged_df = AMZN_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
AMZN_merged_df = AMZN_merged_df.dropna()
AMZN_merged_df.head()

Date   Price    Open    High     Low   Vol.  Change %  \
286 2022-01-04  167.52  170.44  171.40  166.35  70.73     -1.69   
285 2022-01-05  164.36  166.88  167.13  164.36  64.30     -1.89   
284 2022-01-06  163.25  163.45  164.80  161.94  51.96     -0.68   
283 2022-01-07  162.55  163.84  165.24  162.03  46.61     -0.43   
282 2022-01-10  161.49  160.59  161.66  156.30  87.80     -0.65   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             0.500000              1.0             0.277778  ...   
285             0.555556              3.0             0.500000  ...   
284             0.555556              0.0             0.833333  ...   
283             0.375000              2.0             0.833333  ...   
282             0.000000              0.0             0.500000  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                  0.000000                 0.500000   
285                  0.458333                 0.833333   
284                  0.277778                 0.833333   
283                  0.500000                 0.500000   
282                  0.833333                 0.720000   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                 0.833333                   0.0                  3.0   
285                 0.833333                   1.0                  0.0   
284                 0.500000                   3.0                  2.0   
283                 0.720000                   0.0                  0.0   
282                 0.360000                   2.0                  3.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   6.0                  1.0          2.21          0   
285                   3.0                  3.0         -1.69          0   
284                   1.0                  0.0         -1.89          0   
283                   3.0                  1.0         -0.68          0   
282                   0.0                  5.0         -0.43          0   

     lag_1_Vol.  
286       63.87  
285       70.73  
284       64.30  
283       51.96  
282       46.61  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [42]:
import statsmodels.api as sm
X = AMZN_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = AMZN_merged_df['direction']
model_amzn = sm.OLS(y, X).fit()

# Print the model summary
print(model_amzn.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.487
Model:                            OLS   Adj. R-squared (uncentered):              0.456
Method:                 Least Squares   F-statistic:                              15.93
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    1.52e-30
Time:                        09:36:38   Log-Likelihood:                         -201.86
No. Observations:                 285   AIC:                                      435.7
Df Residuals:                     269   BIC:                                      494.2
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### MSFT

In [43]:
MSFT_comment_sentiment = comment_time_series[['Date','MSFT','MSFT_count']]
MSFT_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
MSFT_opinion_sentiment = opinion_time_series[['Date','MSFT','MSFT_count']]
MSFT_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
MSFT_sentiment = pd.merge(MSFT_comment_sentiment, MSFT_opinion_sentiment, on='Date', how='left')
MSFT_sentiment.dropna()
MSFT_price = pd.read_csv('MSFT Historical Data.csv')
MSFT_price['Date'] = pd.to_datetime(MSFT_price['Date'], format='%m/%d/%Y')
MSFT_sentiment['Date'] = pd.to_datetime(MSFT_sentiment['Date'])
merged_df = pd.merge(MSFT_price, MSFT_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
MSFT_merged_df = merged_df.iloc[::-1]
MSFT_merged_df['comment_sentiment_change'] = MSFT_merged_df['comment_sentiment'] - MSFT_merged_df['comment_sentiment'].shift(1)
MSFT_merged_df['opinion_sentiment_change'] = MSFT_merged_df['opinion_sentiment'] - MSFT_merged_df['opinion_sentiment'].shift(1)
MSFT_merged_df['lag_-1_comment_sentiment'] = MSFT_merged_df['comment_sentiment'].shift(1) #前一天的
MSFT_merged_df['lag_-2_comment_sentiment'] = MSFT_merged_df['comment_sentiment'].shift(2)
MSFT_merged_df['lag_1_comment_sentiment'] = MSFT_merged_df['comment_sentiment'].shift(-1)
MSFT_merged_df['lag_2_comment_sentiment'] = MSFT_merged_df['comment_sentiment'].shift(-2)
MSFT_merged_df['lag_-1_opinion_sentiment'] = MSFT_merged_df['opinion_sentiment'].shift(1) #前一天的
MSFT_merged_df['lag_-2_opinion_sentiment'] = MSFT_merged_df['opinion_sentiment'].shift(2)
MSFT_merged_df['lag_1_opinion_sentiment'] = MSFT_merged_df['opinion_sentiment'].shift(-1)
MSFT_merged_df['lag_2_opinion_sentiment'] = MSFT_merged_df['opinion_sentiment'].shift(-2)

MSFT_merged_df['lag_-1_comment_count'] = MSFT_merged_df['comment_count'].shift(1)
MSFT_merged_df['lag_1_comment_count'] = MSFT_merged_df['comment_count'].shift(-1)
MSFT_merged_df['lag_-1_opinion_count'] = MSFT_merged_df['opinion_count'].shift(1)
MSFT_merged_df['lag_1_opinion_count'] = MSFT_merged_df['opinion_count'].shift(-1)

MSFT_merged_df['Change %'] = MSFT_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
MSFT_merged_df['lag_1_change'] = MSFT_merged_df['Change %'].shift(1)
MSFT_merged_df['direction'] = (MSFT_merged_df['Change %'] > 0).astype(int)
MSFT_merged_df['Vol.'] = MSFT_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
MSFT_merged_df['lag_1_Vol.'] = MSFT_merged_df['Vol.'].shift(1)
MSFT_merged_df = MSFT_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
MSFT_merged_df = MSFT_merged_df.rename(columns={'comment_count': '0_comment_count'})
MSFT_merged_df = MSFT_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
MSFT_merged_df = MSFT_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
MSFT_merged_df = MSFT_merged_df.dropna()
MSFT_merged_df.head()

Date   Price    Open    High     Low   Vol.  Change %  \
286 2022-01-04  329.01  334.83  335.20  326.12  32.67     -1.71   
285 2022-01-05  316.38  325.86  326.07  315.98  40.05     -3.84   
284 2022-01-06  313.88  313.15  318.70  311.49  39.65     -0.79   
283 2022-01-07  314.04  314.15  316.50  310.09  32.72      0.05   
282 2022-01-10  314.27  309.48  314.72  304.69  44.29      0.07   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286                0.000              1.0                  0.0  ...   
285                0.375              2.0                  0.5  ...   
284                0.375              0.0                  0.5  ...   
283                0.375              0.0                  0.5  ...   
282                0.000              0.0                  0.5  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                  0.000000                  0.50000   
285                  0.663265                  0.50000   
284                  0.000000                  0.50000   
283                  0.500000                  0.50000   
282                  0.500000                  0.65625   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                  0.50000                   0.0                  2.0   
285                  0.50000                   1.0                  0.0   
284                  0.50000                   2.0                  0.0   
283                  0.65625                   0.0                  0.0   
282                  0.37500                   0.0                  1.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   7.0                  1.0         -0.47          0   
285                   2.0                  0.0         -1.71          0   
284                   1.0                  1.0         -3.84          0   
283                   0.0                  1.0         -0.79          1   
282                   1.0                  4.0          0.05          1   

     lag_1_Vol.  
286       28.92  
285       32.67  
284       40.05  
283       39.65  
282       32.72  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [44]:
X = MSFT_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = MSFT_merged_df['direction']
model_msft = sm.OLS(y, X).fit()

# Print the model summary
print(model_msft.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.504
Model:                            OLS   Adj. R-squared (uncentered):              0.475
Method:                 Least Squares   F-statistic:                              17.09
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    1.78e-32
Time:                        09:36:50   Log-Likelihood:                         -194.75
No. Observations:                 285   AIC:                                      421.5
Df Residuals:                     269   BIC:                                      479.9
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### AAPL

In [45]:
AAPL_comment_sentiment = comment_time_series[['Date','AAPL','AAPL_count']]
AAPL_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
AAPL_opinion_sentiment = opinion_time_series[['Date','AAPL','AAPL_count']]
AAPL_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
AAPL_sentiment = pd.merge(AAPL_comment_sentiment, AAPL_opinion_sentiment, on='Date', how='left')
AAPL_sentiment.dropna()
AAPL_price = pd.read_csv('AAPL Historical Data.csv')
AAPL_price['Date'] = pd.to_datetime(AAPL_price['Date'], format='%m/%d/%Y')
AAPL_sentiment['Date'] = pd.to_datetime(AAPL_sentiment['Date'])
merged_df = pd.merge(AAPL_price, AAPL_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
AAPL_merged_df = merged_df.iloc[::-1]
AAPL_merged_df['comment_sentiment_change'] = AAPL_merged_df['comment_sentiment'] - AAPL_merged_df['comment_sentiment'].shift(1)
AAPL_merged_df['opinion_sentiment_change'] = AAPL_merged_df['opinion_sentiment'] - AAPL_merged_df['opinion_sentiment'].shift(1)
AAPL_merged_df['lag_-1_comment_sentiment'] = AAPL_merged_df['comment_sentiment'].shift(1) #前一天的
AAPL_merged_df['lag_-2_comment_sentiment'] = AAPL_merged_df['comment_sentiment'].shift(2)
AAPL_merged_df['lag_1_comment_sentiment'] = AAPL_merged_df['comment_sentiment'].shift(-1)
AAPL_merged_df['lag_2_comment_sentiment'] = AAPL_merged_df['comment_sentiment'].shift(-2)
AAPL_merged_df['lag_-1_opinion_sentiment'] = AAPL_merged_df['opinion_sentiment'].shift(1) #前一天的
AAPL_merged_df['lag_-2_opinion_sentiment'] = AAPL_merged_df['opinion_sentiment'].shift(2)
AAPL_merged_df['lag_1_opinion_sentiment'] = AAPL_merged_df['opinion_sentiment'].shift(-1)
AAPL_merged_df['lag_2_opinion_sentiment'] = AAPL_merged_df['opinion_sentiment'].shift(-2)

AAPL_merged_df['lag_-1_comment_count'] = AAPL_merged_df['comment_count'].shift(1)
AAPL_merged_df['lag_1_comment_count'] = AAPL_merged_df['comment_count'].shift(-1)
AAPL_merged_df['lag_-1_opinion_count'] = AAPL_merged_df['opinion_count'].shift(1)
AAPL_merged_df['lag_1_opinion_count'] = AAPL_merged_df['opinion_count'].shift(-1)

AAPL_merged_df['Change %'] = AAPL_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
AAPL_merged_df['lag_1_change'] = AAPL_merged_df['Change %'].shift(1)
AAPL_merged_df['direction'] = (AAPL_merged_df['Change %'] > 0).astype(int)
AAPL_merged_df['Vol.'] = AAPL_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
AAPL_merged_df['lag_1_Vol.'] = AAPL_merged_df['Vol.'].shift(1)
AAPL_merged_df = AAPL_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
AAPL_merged_df = AAPL_merged_df.rename(columns={'comment_count': '0_comment_count'})
AAPL_merged_df = AAPL_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
AAPL_merged_df = AAPL_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
AAPL_merged_df = AAPL_merged_df.dropna()
AAPL_merged_df.head()

Date   Price    Open    High     Low    Vol.  Change %  \
286 2022-01-04  179.70  182.63  182.94  179.12   99.31     -1.27   
285 2022-01-05  174.92  179.61  180.17  174.64   94.54     -2.66   
284 2022-01-06  172.00  172.70  175.30  171.64   96.90     -1.67   
283 2022-01-07  172.17  172.89  174.14  171.03   86.71      0.10   
282 2022-01-10  172.19  169.08  172.50  168.17  106.77      0.01   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286             1.222222              6.0               0.3600  ...   
285             0.530612              7.0               0.3750  ...   
284             0.750000              2.0               0.3750  ...   
283             1.222222              6.0               0.3750  ...   
282             0.500000              1.0               1.3125  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                  0.000000                  0.37500   
285                  0.524691                  0.37500   
284                  0.360000                  0.37500   
283                  0.375000                  1.31250   
282                  0.375000                  0.65625   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                  0.37500                   0.0                  7.0   
285                  0.37500                   6.0                  2.0   
284                  1.31250                   7.0                  6.0   
283                  0.65625                   2.0                  1.0   
282                  0.37500                   6.0                  7.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   9.0                  2.0          2.50          0   
285                   5.0                  2.0         -1.27          0   
284                   2.0                  2.0         -2.66          0   
283                   2.0                  4.0         -1.67          1   
282                   2.0                  4.0          0.10          1   

     lag_1_Vol.  
286      104.70  
285       99.31  
284       94.54  
283       96.90  
282       86.71  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power

In [47]:
X = AAPL_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = AAPL_merged_df['direction']
model_aapl = sm.OLS(y, X).fit()

# Print the model summary
print(model_aapl.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.456
Model:                            OLS   Adj. R-squared (uncentered):              0.424
Method:                 Least Squares   F-statistic:                              14.12
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    2.09e-27
Time:                        09:37:09   Log-Likelihood:                         -215.21
No. Observations:                 285   AIC:                                      462.4
Df Residuals:                     269   BIC:                                      520.9
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### AMC

In [48]:
AMC_comment_sentiment = comment_time_series[['Date','AMC','AMC_count']]
AMC_comment_sentiment.columns = ['Date','comment_sentiment','comment_count']
AMC_opinion_sentiment = opinion_time_series[['Date','AMC','AMC_count']]
AMC_opinion_sentiment.columns = ['Date','opinion_sentiment','opinion_count']
AMC_sentiment = pd.merge(AMC_comment_sentiment, AMC_opinion_sentiment, on='Date', how='left')
AMC_sentiment.dropna()
AMC_price = pd.read_csv('AMC Historical Data.csv')
AMC_price['Date'] = pd.to_datetime(AMC_price['Date'], format='%m/%d/%Y')
AMC_sentiment['Date'] = pd.to_datetime(AMC_sentiment['Date'])
merged_df = pd.merge(AMC_price, AMC_sentiment, on='Date', how='left')
merged_df = merged_df.fillna(0)
AMC_merged_df = merged_df.iloc[::-1]
AMC_merged_df['comment_sentiment_change'] = AMC_merged_df['comment_sentiment'] - AMC_merged_df['comment_sentiment'].shift(1)
AMC_merged_df['opinion_sentiment_change'] = AMC_merged_df['opinion_sentiment'] - AMC_merged_df['opinion_sentiment'].shift(1)
AMC_merged_df['lag_-1_comment_sentiment'] = AMC_merged_df['comment_sentiment'].shift(1) #前一天的
AMC_merged_df['lag_-2_comment_sentiment'] = AMC_merged_df['comment_sentiment'].shift(2)
AMC_merged_df['lag_1_comment_sentiment'] = AMC_merged_df['comment_sentiment'].shift(-1)
AMC_merged_df['lag_2_comment_sentiment'] = AMC_merged_df['comment_sentiment'].shift(-2)
AMC_merged_df['lag_-1_opinion_sentiment'] = AMC_merged_df['opinion_sentiment'].shift(1) #前一天的
AMC_merged_df['lag_-2_opinion_sentiment'] = AMC_merged_df['opinion_sentiment'].shift(2)
AMC_merged_df['lag_1_opinion_sentiment'] = AMC_merged_df['opinion_sentiment'].shift(-1)
AMC_merged_df['lag_2_opinion_sentiment'] = AMC_merged_df['opinion_sentiment'].shift(-2)

AMC_merged_df['lag_-1_comment_count'] = AMC_merged_df['comment_count'].shift(1)
AMC_merged_df['lag_1_comment_count'] = AMC_merged_df['comment_count'].shift(-1)
AMC_merged_df['lag_-1_opinion_count'] = AMC_merged_df['opinion_count'].shift(1)
AMC_merged_df['lag_1_opinion_count'] = AMC_merged_df['opinion_count'].shift(-1)

AMC_merged_df['Change %'] = AMC_merged_df['Change %'].str.strip().str.rstrip('%').astype(float)
AMC_merged_df['lag_1_change'] = AMC_merged_df['Change %'].shift(1)
AMC_merged_df['direction'] = (AMC_merged_df['Change %'] > 0).astype(int)
AMC_merged_df['Vol.'] = AMC_merged_df['Vol.'].str.strip().str.rstrip('M').astype(float)
AMC_merged_df['lag_1_Vol.'] = AMC_merged_df['Vol.'].shift(1)
AMC_merged_df = AMC_merged_df.rename(columns={'comment_sentiment': '0_comment_sentiment'})
AMC_merged_df = AMC_merged_df.rename(columns={'comment_count': '0_comment_count'})
AMC_merged_df = AMC_merged_df.rename(columns={'opinion_sentiment': '0_opinion_sentiment'})
AMC_merged_df = AMC_merged_df.rename(columns={'opinion_count': '0_opinion_count'})
AMC_merged_df = AMC_merged_df.dropna()
AMC_merged_df.head()

Date  Price   Open   High    Low   Vol.  Change %  \
286 2022-01-04  25.49  26.67  26.67  24.64  33.14     -3.88   
285 2022-01-05  22.75  25.17  25.30  22.36  44.65    -10.75   
284 2022-01-06  22.46  22.96  23.77  20.80  51.59     -1.27   
283 2022-01-07  22.99  23.61  24.30  22.44  49.48      2.36   
282 2022-01-10  22.78  22.42  22.87  21.25  37.58     -0.91   

     0_comment_sentiment  0_comment_count  0_opinion_sentiment  ...  \
286                  0.0              0.0                  0.5  ...   
285                  0.0              0.0                  0.5  ...   
284                  0.0              0.0                  0.5  ...   
283                  0.0              0.0                  0.5  ...   
282                  0.0              0.0                  0.5  ...   

     lag_-2_opinion_sentiment  lag_1_opinion_sentiment  \
286                       0.0                      0.5   
285                       0.5                      0.5   
284                       0.5                      0.5   
283                       0.5                      0.5   
282                       0.5                      0.5   

     lag_2_opinion_sentiment  lag_-1_comment_count  lag_1_comment_count  \
286                      0.5                   0.0                  0.0   
285                      0.5                   0.0                  0.0   
284                      0.5                   0.0                  0.0   
283                      0.5                   0.0                  0.0   
282                      0.5                   0.0                  0.0   

     lag_-1_opinion_count  lag_1_opinion_count  lag_1_change  direction  \
286                   1.0                  0.0         -2.50          0   
285                   0.0                  0.0         -3.88          0   
284                   0.0                  0.0        -10.75          0   
283                   0.0                  0.0         -1.27          1   
282                   0.0                  0.0          2.36          0   

     lag_1_Vol.  
286       26.62  
285       33.14  
284       44.65  
283       51.59  
282       49.48  

[5 rows x 28 columns]

##### 验证sentiment具备prediction power¶

In [49]:
X = AMC_merged_df[['lag_-2_comment_sentiment','lag_-1_comment_sentiment', '0_comment_sentiment','lag_1_comment_sentiment',
       'lag_2_comment_sentiment', 'lag_-2_opinion_sentiment', 'lag_-1_opinion_sentiment', '0_opinion_sentiment','lag_1_opinion_sentiment',
       'lag_2_opinion_sentiment','lag_-1_opinion_count','0_opinion_count','lag_1_opinion_count',
                    'lag_-1_comment_count','0_comment_count','lag_1_comment_count']]
# X = sm.add_constant(X)  
y = AMC_merged_df['direction']
model_amc = sm.OLS(y, X).fit()

# Print the model summary
print(model_amc.summary())


                                 OLS Regression Results                                
Dep. Variable:              direction   R-squared (uncentered):                   0.290
Model:                            OLS   Adj. R-squared (uncentered):              0.247
Method:                 Least Squares   F-statistic:                              6.857
Date:                Sun, 23 Apr 2023   Prob (F-statistic):                    3.96e-13
Time:                        09:37:17   Log-Likelihood:                         -233.57
No. Observations:                 285   AIC:                                      499.1
Df Residuals:                     269   BIC:                                      557.6
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

#### Others

In [50]:
# List of stocks to keep
stocks_to_keep = ['TSLA', 'LCID', 'META', 'AMZN', 'MSFT', 'AAPL', 'AMC', 'AMD', 'BABA', 'NIO']

# Filter the DataFrame to keep only the stocks in the list
comments = sentiment_time_series[stocks_to_keep]

# Reset the index of the filtered DataFrame (optional)
#comments.reset_index(drop=True, inplace=True)

# Show the filtered DataFrame
print(comments)


NameError: name 'sentiment_time_series' is not defined

In [29]:
null_counts = comments.isnull().sum()
print(null_counts)

stock
TSLA     19
LCID    322
META    193
AMZN     96
MSFT    305
AAPL    166
AMC     307
AMD     175
BABA    207
NIO      59
dtype: int64


In [54]:
stargazer_tab = Stargazer([model_tesla, model_nio, model_meta, model_amzn, model_msft])
# Using a Python interactive window, simply run the 
# name of the object to display the table
stargazer_tab

In [56]:
from stargazer.stargazer import Stargazer

stargazer_tab = Stargazer([model_baba, model_amd, model_lcid, model_aapl, model_amc])
# Using a Python interactive window, simply run the 
# name of the object to display the table
stargazer_tab


In [57]:
open('regression.tex', 'w').write(stargazer_tab.render_latex())  # for latex
open('regression.html', 'w').write(stargazer_tab.render_html())  # for html

6535

In [53]:
stargazer_tab.render_latex()

'\\begin{table}[!htbp] \\centering\n\\begin{tabular}{@{\\extracolsep{5pt}}lcccccccccc}\n\\\\[-1.8ex]\\hline\n\\hline \\\\[-1.8ex]\n& \\multicolumn{10}{c}{\\textit{Dependent variable:}} \\\n\\cr \\cline{10-11}\n\\\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) & (10) \\\\\n\\hline \\\\[-1.8ex]\n 0_comment_count & 0.001$^{}$ & -0.001$^{}$ & 0.020$^{*}$ & -0.017$^{}$ & -0.003$^{}$ & -0.001$^{}$ & -0.001$^{}$ & 0.029$^{**}$ & 0.014$^{}$ & 0.010$^{}$ \\\\\n  & (0.001) & (0.003) & (0.011) & (0.011) & (0.003) & (0.035) & (0.002) & (0.013) & (0.012) & (0.024) \\\\\n 0_comment_sentiment & 0.094$^{}$ & 0.117$^{}$ & 0.002$^{}$ & 0.155$^{*}$ & 0.217$^{***}$ & 0.119$^{}$ & 0.147$^{*}$ & 0.104$^{}$ & 0.038$^{}$ & 0.055$^{}$ \\\\\n  & (0.124) & (0.090) & (0.080) & (0.089) & (0.081) & (0.122) & (0.075) & (0.114) & (0.074) & (0.134) \\\\\n 0_opinion_count & 0.013$^{}$ & -0.035$^{}$ & 0.123$^{*}$ & 0.032$^{}$ & -0.015$^{}$ & 0.034$^{}$ & 0.005$^{}$ & 0.027$^{}$ & -0.031$^{}$ & 0.181$^{}

In [58]:
df = pd.read_csv('df_all_predict_0422.csv', encoding='ISO-8859-1')
df

stock        Date                                               text  \
0      BABA  2023/01/31  China's reopening has revived its economy but ...   
1      BABA  2023/01/26  Investors seem to be positioning for a strong ...   
2      BABA  2023/01/17  E-commerce behemoth Alibaba Group Holdings (NY...   
3      BABA  2023/01/13  Alibaba shares are up 100% from their lows. Ta...   
4      BABA  2023/01/13  The continued easing of US price pressures has...   
...     ...         ...                                                ...   
26063   NIO  2022/07/15  #TSLA Held 20 Day SMA today and closed above 7...   
26064  TSLA  2022/02/24                     750 as I wrote 10 hours ago...   
26065  TSLA  2022/11/10                           Crypto will push it down   
26066  TSLA  2022/03/04  This really doesnt want to move like the other...   
26067  TSLA  2022/04/25                                  Just can't sleep.   

      category label  
0            2     1  
1            2     1  
2            2     1  
3            2     1  
4            2     0  
...        ...   ...  
26063        1     1  
26064        1     1  
26065        1     2  
26066        1     0  
26067        1     0  

[26068 rows x 5 columns]